In [1]:
# Installing packages in the notebook for conda if needed
# %pip install mne
# %pip install statsmodels
# %pip install scikit-learn
# %pip install researchpy
# 

In [2]:
# importing modules
import os
import pip
import numpy as np
import pandas as pd
import mne
from scipy import stats as st
import statistics as stats
from itertools import combinations
import researchpy as rp
from statsmodels.stats.multitest import multipletests as multi

In [3]:
epochs = mne.read_epochs('./data/Epochs/thatcher_epochs.fif')

Reading c:\Users\lauri\OneDrive - Aarhus universitet\Dokumenter\EEG_Thatcher_Effect\data\Epochs\thatcher_epochs.fif ...
    Found the data of interest:
        t =    -100.00 ...     696.00 ms
        0 CTF compensation matrices available
Not setting metadata
358 matching events found
No baseline correction applied
0 projection items activated


C:\Users\lauri\AppData\Local\Temp\ipykernel_20676\2480100391.py:1: RuntimeWarning: This filename (./data/Epochs/thatcher_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs('./data/Epochs/thatcher_epochs.fif')


In [4]:
# NOT evoked variabled 
all_changed = epochs['Changed']
all_unchanged = epochs['Unchanged']

# Not evoked Interactions
changed_0 = epochs['Changed/0']
changed_90 = epochs['Changed/90']
changed_180 = epochs['Changed/180']

unchanged_0 = epochs['Unchanged/0']
unchanged_90 = epochs['Unchanged/90']
unchanged_180 = epochs['Unchanged/180']



# Rotation Epochs:
# Not evoked Interactions
epoch_0 = epochs['0']
epoch_90 = epochs['90']
epoch_180 = epochs['180']




# Evoked variables
## Changed vs unchaged
all_changed_evoked = epochs['Changed'].average()
all_unchanged_evoked = epochs['Unchanged'].average()

# INTERACTIONS
# Changed vs Unchanged X Inversion effect
## creating for changed vs unchanged for each rotation
# changed_0 = epochs['Changed/0'].average()
# changed_90 = epochs['Changed/90'].average()
# changed_180 = epochs['Changed/180'].average()

# unchanged_0 = epochs['Unchanged/0'].average()
# unchanged_90 = epochs['Unchanged/90'].average()
# unchanged_180 = epochs['Unchanged/180'].average()

# Familiar vs Unfamiliar X Changed vs Unchanged X Inversion effect
## creating for famiar vs unfamiliar interacting with inverse effect

# ## Familiar
# changed_0_familiar = epochs['Changed/0/Familliar'].average()
# changed_90_familiar = epochs['Changed/90/Familliar'].average()
# changed_180_familiar = epochs['Changed/180/Familliar'].average()

# unchanged_0_familiar = epochs['Unchanged/0/Familliar'].average()
# unchanged_90_familiar = epochs['Unchanged/90/Familliar'].average()
# unchanged_180_familiar = epochs['Unchanged/180/Familliar'].average()

# ## Unfamiliar
# changed_0_unfamiliar = epochs['Changed/0/Unfamilliar'].average()
# changed_90_unfamiliar = epochs['Changed/90/Unfamilliar'].average()
# changed_180_unfamiliar = epochs['Changed/180/Unfamilliar'].average()

# unchanged_0_unfamiliar = epochs['Unchanged/0/Unfamilliar'].average()
# unchanged_90_unfamiliar = epochs['Unchanged/90/Unfamilliar'].average()
# unchanged_180_unfamiliar = epochs['Unchanged/180/Unfamilliar'].average()

# Windowed mean
One of the simplest way in which we can determine whether the signal in our two conditions are statistically significant is by:

Segmenting our data using only certain channels in a specific time window. Keep in mind that which time window and channels should be established a priori, for instance according to the literature.
Taking the mean of that window across channels and and samples.
Running statistical tests on the windowed means from the two conditions.
In an experiment with multiple participants we would also average over trials from individual participants, in order to only have one data point per participant (and thereby avoid multiple comparisons). However, since we have one participant, we can keep one dimension of the individual data, i.e. the trials.

## T-test
We can now do a t-test on the trials from the two conditions, to establish whether the means of the two groups are statistically significant.

We can use the get_data() function to get the numerical values of the signal (in microvolts) for the t-test. tmin and tmax are used to define the size of the window, and the picks are the channels that we expect to see an effect in.


## Create ANOVA Test Function

In [5]:
from bioinfokit.analys import stat
def Anova_Epochs(Epoch_Condition, tmin, tmax, channels):
    n_conditions = range(len(Epoch_Condition))
    data = []
    for i in n_conditions:
        data.append(Epoch_Condition[i].get_data(picks=channels, tmin=tmin, tmax=tmax))
    
    means = []
    for i in n_conditions:
        mean_temp = np.mean(data[i], axis=2)
        means.append(np.mean(mean_temp, axis=1))
    
    res = stat()
    return st.f_oneway(*means)

# N170
## 'P3', 'P7', "P4", "P8", "O1", "Oz", "O2"

In [6]:
print(Anova_Epochs(Epoch_Condition = [changed_180, unchanged_180], 
             tmin = .160, 
             tmax = .180,
             channels = ["P7", "P8", "TP9", "TP10", "P9", "P10"]))### Default Threshold 

F_onewayResult(statistic=2.1727087727850023, pvalue=0.14588584520515363)


C:\Users\lauri\AppData\Local\Temp\ipykernel_20676\3370433452.py:6: RuntimeWarning: Channel(s) ['P9', 'P10'] could not be picked, because they are not present in the info instance.
  data.append(Epoch_Condition[i].get_data(picks=channels, tmin=tmin, tmax=tmax))
C:\Users\lauri\AppData\Local\Temp\ipykernel_20676\3370433452.py:6: RuntimeWarning: Channel(s) ['P9', 'P10'] could not be picked, because they are not present in the info instance.
  data.append(Epoch_Condition[i].get_data(picks=channels, tmin=tmin, tmax=tmax))


In [7]:
print(Anova_Epochs(Epoch_Condition = [changed_0, unchanged_0], 
             tmin = .160, 
             tmax = .180,
             channels = ["P7", "P8", "TP9", "TP10", "P9", "P10"]))### Default Threshold 

F_onewayResult(statistic=1.0793747198866743, pvalue=0.3031493051450015)


C:\Users\lauri\AppData\Local\Temp\ipykernel_20676\3370433452.py:6: RuntimeWarning: Channel(s) ['P9', 'P10'] could not be picked, because they are not present in the info instance.
  data.append(Epoch_Condition[i].get_data(picks=channels, tmin=tmin, tmax=tmax))
C:\Users\lauri\AppData\Local\Temp\ipykernel_20676\3370433452.py:6: RuntimeWarning: Channel(s) ['P9', 'P10'] could not be picked, because they are not present in the info instance.
  data.append(Epoch_Condition[i].get_data(picks=channels, tmin=tmin, tmax=tmax))


## N170
"P7", "P8", "TP9", "TP10", "P9", "P10"

### Create T-Test Function

In [8]:
def Ttest_Epochs(Epoch_Condition, tmin, tmax, channels):
    data = []
    for i in range(2):
        data.append(Epoch_Condition[i].get_data(picks=channels, tmin=tmin, tmax=tmax))
    
    means = []
    for i in range(2):
        mean_temp = np.mean(data[i], axis=2)
        means.append(np.mean(mean_temp, axis=1))
    
    return st.ttest_ind(a=means[0], b=means[1])  

In [9]:
print(Ttest_Epochs(Epoch_Condition = [all_changed, all_unchanged], 
             tmin = .160, 
             tmax = .180,
             channels = ["P3", "P7", "P4", "P8", "O1", "Oz", "O2"]))

Ttest_indResult(statistic=0.03918748972513657, pvalue=0.9687848247554656)


### Pairwise T-tests in The Significant Window

In [10]:
def pairwise_Ttest(Epoch_Condition, tmin, tmax, channels, Epoch_Names):
    n_conditions = range(len(Epoch_Condition))
    data = []
    for i in n_conditions:
        data.append(Epoch_Condition[i].get_data(picks=channels, tmin=tmin, tmax=tmax))
    
    means = []
    for i in n_conditions:
        mean_temp = np.mean(data[i], axis=2)
        means.append(np.mean(mean_temp, axis=1))
    
    ## create empty lists
    pairwise_names = []
    pairwise_pval = []
    pairwise_tstat = []
    
    ## make all the combinations
    for a, b in combinations(Epoch_Names, 2): 
        y = f"{a} vs. {b}"
        pairwise_names.append(y)
        
    for a, b in combinations(means, 2): # function which make all combinations of 2 means possible
        y = st.ttest_ind(a, b)
        pairwise_pval.append(y.pvalue)
        pairwise_tstat.append(y.statistic)
    
    
    bonferroni = multi(pairwise_pval, alpha=0.05, method='bonferroni', is_sorted=False, returnsorted=False)
    
    ## create dataframe
    data = pd.DataFrame({"Comparison":pairwise_names, "P-Value":pairwise_pval, "t-statistic":pairwise_tstat, "Bonferroni Adj.":bonferroni[1]})
    
    return data
    

In [ ]:
## N170
pairwise = pairwise_Ttest(Epoch_Condition = [epoch_0, epoch_90, epoch_180], 
             tmin = .160, 
             tmax = .180,
             channels = ["P7", "P8", "TP9", "TP10", "P9", "P10"],
             Epoch_Names = ["Normal", "Ninety", "Rotated"]
              )

pairwise

,Comparison,P-Value,t-statistic,Bonferroni Adj.
0,Normal vs. Ninety,0.814242,0.235483,1.0
1,Normal vs. Rotated,0.859431,0.177486,1.0
2,Ninety vs. Rotated,0.978160,-0.027433,1.0


In [ ]:
## N170
pairwise = pairwise_Ttest(Epoch_Condition = [epoch_0, epoch_90, epoch_180], 
             tmin = .160, 
             tmax = .180,
             channels = ["P7", "P8", "TP9", "TP10", "P9", "P10"],
             Epoch_Names = ["Normal", "Ninety", "Rotated"]
              )

pairwise

,Comparison,P-Value,t-statistic,Bonferroni Adj.
0,Normal vs. Ninety,0.226410,-1.216005,0.679231
1,Normal vs. Rotated,0.643502,-0.464000,1.000000
2,Ninety vs. Rotated,0.460567,0.740335,1.000000
